In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dacon-data/sample_submission.csv
/kaggle/input/dacon-data/train.csv
/kaggle/input/dacon-data/test.csv


# Introductory Summary

1. Rank : 47th / 1,116 (Competition participated)

2. Preprocessing : Label Encoding 

3. Feature Engineering : Statistics Aggregation 

4. Model : CatBoost

In [2]:
train = pd.read_csv('/kaggle/input/dacon-data/train.csv')
test = pd.read_csv('/kaggle/input/dacon-data/test.csv')

In [3]:
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,...,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,...,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [4]:
test

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,TEST_0000,COMPONENT1,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,TEST_0001,COMPONENT3,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,TEST_0002,COMPONENT2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,TEST_0003,COMPONENT3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,TEST_0004,COMPONENT2,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,TEST_6037,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,TEST_6038,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,TEST_6039,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [5]:
train['Y_LABEL'].value_counts() #0,1 비율확인

0    12892
1     1203
Name: Y_LABEL, dtype: int64

In [6]:
pd.options.display.max_columns = 99
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0,1,0,0,1,0,572,2,19,1117,0,0,0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0,4,1,0,0,0,906,1,44,21774,2,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,6,0.0,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,0.0,1.0,0,0,0,0,8,0,477,0,35,22000,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,2945,0.0,0,0,2,13.0,8.0,0.0,16.0,21.0,9.1,12,0.0,0.0,4.0,0,36,0,224,3,0,1163,1,9,5494,0,5,0,0.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,13,0.0,0,4,7,NaN,NaN,NaN,NaN,NaN,NaN,415,NaN,0.0,NaN,1,16,7,10,3,1,1175,1,645,9640,2,18,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,2244,NaN,0,0,170,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,0.0,0.0,0,0,0,1,0,0,441,1,11,1313,0,0,0,NaN,0,0.0,0.0,0.0,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [7]:
alldata = pd.concat([train,test])
alldata

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7.0,200,0,3.0,93.0,0.0,0.0,3059.0,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1.0,14.0,16,1,16.0,6,1951.0,0.0,8504,21370.0,0.0,427.0,2.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0.0
1,TRAIN_00001,COMPONENT2,1350,2021,51.0,375,0,2.0,19.0,0.0,0.0,2978.0,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0.0,1.0,0,0,1.0,0,572.0,2.0,19,1117.0,0.0,0.0,0.0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0.0
2,TRAIN_00002,COMPONENT2,2415,2015,2.0,200,0,110.0,1.0,1.0,0.0,17.0,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0.0,0.0,0,0,2.0,0,328.0,0.0,17,1334.0,0.0,0.0,1.0,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1.0
3,TRAIN_00003,COMPONENT3,7389,2010,2.0,200,0,8.0,3.0,0.0,0.0,1960.0,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0.0,4.0,1,0,0.0,0,906.0,1.0,44,21774.0,2.0,1.0,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0.0
4,TRAIN_00004,COMPONENT3,3954,2015,4.0,200,0,1.0,157.0,0.0,0.0,71.0,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0.0,0.0,0,0,2.0,0,309.0,0.0,217,18470.0,0.0,2.0,0.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,TEST_6036,COMPONENT3,1714,2014,NaN,200,0,NaN,NaN,NaN,NaN,NaN,NaN,0,3,130,NaN,NaN,NaN,NaN,NaN,NaN,1047,NaN,0.0,NaN,NaN,NaN,65,5,NaN,2,NaN,NaN,1999,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,71.2,1163,NaN
6037,TEST_6037,COMPONENT3,4131,2016,NaN,200,0,NaN,NaN,NaN,NaN,NaN,NaN,0,5,2,NaN,NaN,NaN,NaN,NaN,NaN,736,NaN,0.0,NaN,NaN,NaN,5,0,NaN,0,NaN,NaN,693,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,137.8,13,NaN
6038,TEST_6038,COMPONENT3,4325,2014,NaN,200,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,53,NaN,0.0,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,69,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,302.2,13,NaN
6039,TEST_6039,COMPONENT2,1364,2013,NaN,200,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,62,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,NaN,NaN,0,0,NaN,0,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,51.1,478,NaN


In [8]:
test.columns

Index(['ID', 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
       'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
       'V40', 'ZN'],
      dtype='object')

In [9]:
train

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,BE,CA,CD,CO,CR,CU,FH2O,FNOX,FOPTIMETHGLY,FOXID,FSO4,FTBN,FE,FUEL,H2O,K,LI,MG,MN,MO,NA,NI,P,PB,PQINDEX,S,SB,SI,SN,SOOTPERCENTAGE,TI,U100,U75,U50,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,0,3059,0.0,0,13,78,NaN,NaN,NaN,NaN,NaN,NaN,888,NaN,0.0,27.0,1,14,16,1,16,6,1951,0,8504,21370,0,427,2,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,0,2978,0.0,0,0,31,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,0.0,NaN,0,1,0,0,1,0,572,2,19,1117,0,0,0,NaN,0,0.0,0.0,1.0,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,0,17,0.0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,0.0,0.0,0,0,0,0,2,0,328,0,17,1334,0,0,1,NaN,0,0.0,0.0,0.0,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,0,1960,0.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,37,NaN,0.0,NaN,0,4,1,0,0,0,906,1,44,21774,2,1,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,0,71,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,71,NaN,0.0,0.0,0,0,0,0,2,0,309,0,217,18470,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,TRAIN_14090,COMPONENT3,1616,2014,8,200,0,2,201,1,0,6,0.0,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,0.0,1.0,0,0,0,0,8,0,477,0,35,22000,0,2,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,135.4,16,0
14091,TRAIN_14091,COMPONENT1,2784,2013,2,200,0,3,85,0,0,2945,0.0,0,0,2,13.0,8.0,0.0,16.0,21.0,9.1,12,0.0,0.0,4.0,0,36,0,224,3,0,1163,1,9,5494,0,5,0,0.4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,14.5,117.5,1408,0
14092,TRAIN_14092,COMPONENT3,1788,2008,9,550,0,6,0,1,0,13,0.0,0,4,7,NaN,NaN,NaN,NaN,NaN,NaN,415,NaN,0.0,NaN,1,16,7,10,3,1,1175,1,645,9640,2,18,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,54.0,1301,0
14093,TRAIN_14093,COMPONENT2,2498,2009,19,550,0,2,4,0,0,2244,NaN,0,0,170,NaN,NaN,NaN,NaN,NaN,NaN,19,NaN,0.0,0.0,0,0,0,1,0,0,441,1,11,1313,0,0,0,NaN,0,0.0,0.0,0.0,7.0,8.0,100.0,1625.0,18890.0,0,NaN,44.3,652,0


In [10]:
alldata2 = alldata[test.columns[1:]] ###train + test 합친 데이터에서 test 컬럼만 추출(id 컬럼 제외)
alldata2

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,COMPONENT3,1486,2011,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75
1,COMPONENT2,1350,2021,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652
2,COMPONENT2,2415,2015,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412
3,COMPONENT3,7389,2010,200,0,0,0,1,37,0.0,1,0,0,44,0,0,133.3,7
4,COMPONENT3,3954,2015,200,0,0,0,0,71,0.0,0,0,0,217,0,0,133.1,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [11]:
# com_ano1 = alldata2.groupby('COMPONENT_ARBITRARY')['ANONYMOUS_1'].mean()
com_ano1 = alldata2.groupby('COMPONENT_ARBITRARY')['ANONYMOUS_1'].agg(['mean', 'median', 'max', 'std', 'sum','count'])
com_ano1

,mean,median,max,std,sum,count
COMPONENT_ARBITRARY,,,,,,
COMPONENT1,3029.773079,2206.0,71569,2879.562337,16836449,5557
COMPONENT2,3211.828045,2236.0,239570,5913.051725,10627939,3309
COMPONENT3,3199.066627,2248.0,294451,4558.576706,32217800,10071
COMPONENT4,3325.989158,2377.0,67013,4145.189269,3987861,1199


In [12]:
year_ano2 = alldata2.groupby('YEAR')['ANONYMOUS_2'].agg(['mean', 'median', 'max', 'std', 'sum','count'])
year_ano2

,mean,median,max,std,sum,count
YEAR,,,,,,
2007,456.733475,375.0,1727,282.492779,214208,469
2008,440.698866,375.0,2029,286.245987,894178,2029
2009,325.594955,200.0,3700,293.264767,438902,1348
2010,306.509120,200.0,3702,246.456965,386508,1261
2011,278.657976,200.0,6524,268.132685,448918,1611
2012,273.880859,200.0,2582,222.608824,420681,1536
2013,257.651822,200.0,2336,202.289572,445480,1729
2014,254.571937,200.0,3864,218.343954,500743,1967
2015,265.962356,200.0,3703,213.738433,438040,1647


In [13]:
alldata2 = pd.merge(alldata2,com_ano1,on = 'COMPONENT_ARBITRARY', how = 'left')
alldata2 = pd.merge(alldata2,year_ano2,on = 'YEAR', how = 'left')
alldata2

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,mean_x,median_x,max_x,std_x,sum_x,count_x,mean_y,median_y,max_y,std_y,sum_y,count_y
0,COMPONENT3,1486,2011,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75,3199.066627,2248.0,294451,4558.576706,32217800,10071,278.657976,200.0,6524,268.132685,448918,1611
1,COMPONENT2,1350,2021,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652,3211.828045,2236.0,239570,5913.051725,10627939,3309,558.237263,469.0,3702,403.537471,383509,687
2,COMPONENT2,2415,2015,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412,3211.828045,2236.0,239570,5913.051725,10627939,3309,265.962356,200.0,3703,213.738433,438040,1647
3,COMPONENT3,7389,2010,200,0,0,0,1,37,0.0,1,0,0,44,0,0,133.3,7,3199.066627,2248.0,294451,4558.576706,32217800,10071,306.509120,200.0,3702,246.456965,386508,1261
4,COMPONENT3,3954,2015,200,0,0,0,0,71,0.0,0,0,0,217,0,0,133.1,128,3199.066627,2248.0,294451,4558.576706,32217800,10071,265.962356,200.0,3703,213.738433,438040,1647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20131,COMPONENT3,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163,3199.066627,2248.0,294451,4558.576706,32217800,10071,254.571937,200.0,3864,218.343954,500743,1967
20132,COMPONENT3,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13,3199.066627,2248.0,294451,4558.576706,32217800,10071,294.182945,200.0,6860,395.125519,403619,1372
20133,COMPONENT3,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13,3199.066627,2248.0,294451,4558.576706,32217800,10071,254.571937,200.0,3864,218.343954,500743,1967
20134,COMPONENT2,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478,3211.828045,2236.0,239570,5913.051725,10627939,3309,257.651822,200.0,2336,202.289572,445480,1729


In [14]:
alldata2['COMPONENT_ARBITRARY'].value_counts()

COMPONENT3    10071
COMPONENT1     5557
COMPONENT2     3309
COMPONENT4     1199
Name: COMPONENT_ARBITRARY, dtype: int64

In [15]:
#component_arbitrary 컬럼 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
alldata2['COMPONENT_ARBITRARY'] = le.fit_transform(alldata2['COMPONENT_ARBITRARY'])

In [16]:
alldata2['COMPONENT_ARBITRARY'].value_counts()

2    10071
0     5557
1     3309
3     1199
Name: COMPONENT_ARBITRARY, dtype: int64

In [17]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]
train2

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,mean_x,median_x,max_x,std_x,sum_x,count_x,mean_y,median_y,max_y,std_y,sum_y,count_y
0,2,1486,2011,200,0,0,13,78,888,0.0,16,1,6,8504,5,0,154.0,75,3199.066627,2248.0,294451,4558.576706,32217800,10071,278.657976,200.0,6524,268.132685,448918,1611
1,1,1350,2021,375,0,0,0,31,2,0.0,0,0,0,19,0,0,44.0,652,3211.828045,2236.0,239570,5913.051725,10627939,3309,558.237263,469.0,3702,403.537471,383509,687
2,1,2415,2015,200,0,0,1,2,4,0.0,0,0,0,17,0,0,72.6,412,3211.828045,2236.0,239570,5913.051725,10627939,3309,265.962356,200.0,3703,213.738433,438040,1647
3,2,7389,2010,200,0,0,0,1,37,0.0,1,0,0,44,0,0,133.3,7,3199.066627,2248.0,294451,4558.576706,32217800,10071,306.509120,200.0,3702,246.456965,386508,1261
4,2,3954,2015,200,0,0,0,0,71,0.0,0,0,0,217,0,0,133.1,128,3199.066627,2248.0,294451,4558.576706,32217800,10071,265.962356,200.0,3703,213.738433,438040,1647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14090,2,1616,2014,200,0,0,0,3,23,0.0,0,0,0,35,0,0,135.4,16,3199.066627,2248.0,294451,4558.576706,32217800,10071,254.571937,200.0,3864,218.343954,500743,1967
14091,0,2784,2013,200,0,0,0,2,12,0.0,0,224,0,9,0,0,117.5,1408,3029.773079,2206.0,71569,2879.562337,16836449,5557,257.651822,200.0,2336,202.289572,445480,1729
14092,2,1788,2008,550,0,0,4,7,415,0.0,7,10,1,645,0,0,54.0,1301,3199.066627,2248.0,294451,4558.576706,32217800,10071,440.698866,375.0,2029,286.245987,894178,2029
14093,1,2498,2009,550,0,0,0,170,19,0.0,0,1,0,11,0,0,44.3,652,3211.828045,2236.0,239570,5913.051725,10627939,3309,325.594955,200.0,3700,293.264767,438902,1348


In [18]:
###train2 데이터 중 train,test split 8:2 *10회 반복
###theshold 최적값 반복문

# for i in range(10):
#     from sklearn.model_selection import train_test_split
#     x_train,x_valid,y_train,y_valid = train_test_split(train2,train['Y_LABEL'],test_size = 0.2, random_state = i)


#     from catboost import CatBoostClassifier
#     cbc = CatBoostClassifier(verbose = 100)
#     cbc.fit(x_train,y_train)
#     #cbc.fit(x_train,y_train,eval_set = (x_valid,y_valid))
#     result_sample = cbc.predict_proba(x_valid)
    
#     threshold = np.arange(0.145, 0.155, 0.001)
    
#     for i in threshold:
#         result1 = (result_sample[:,1]>i).astype(int)
#         result1

#         from sklearn.metrics import f1_score
#         print(f1_score(y_valid,result1, average='macro'), i)

In [19]:
test2

,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,mean_x,median_x,max_x,std_x,sum_x,count_x,mean_y,median_y,max_y,std_y,sum_y,count_y
14095,0,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091,3029.773079,2206.0,71569,2879.562337,16836449,5557,294.182945,200.0,6860,395.125519,403619,1372
14096,2,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12,3199.066627,2248.0,294451,4558.576706,32217800,10071,278.657976,200.0,6524,268.132685,448918,1611
14097,1,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714,3211.828045,2236.0,239570,5913.051725,10627939,3309,306.509120,200.0,3702,246.456965,386508,1261
14098,2,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94,3199.066627,2248.0,294451,4558.576706,32217800,10071,325.594955,200.0,3700,293.264767,438902,1348
14099,1,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469,3211.828045,2236.0,239570,5913.051725,10627939,3309,257.651822,200.0,2336,202.289572,445480,1729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20131,2,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163,3199.066627,2248.0,294451,4558.576706,32217800,10071,254.571937,200.0,3864,218.343954,500743,1967
20132,2,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13,3199.066627,2248.0,294451,4558.576706,32217800,10071,294.182945,200.0,6860,395.125519,403619,1372
20133,2,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13,3199.066627,2248.0,294451,4558.576706,32217800,10071,254.571937,200.0,3864,218.343954,500743,1967
20134,1,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478,3211.828045,2236.0,239570,5913.051725,10627939,3309,257.651822,200.0,2336,202.289572,445480,1729


In [20]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 100)
result2 = 0

for train_index, valid_index in skf.split(train2,train['Y_LABEL']):
    x_train = train2.iloc[train_index]
    x_valid = train2.iloc[valid_index]
    y_train = train['Y_LABEL'].iloc[train_index]
    y_valid = train['Y_LABEL'].iloc[valid_index]

    cbc = CatBoostClassifier(verbose = 100)
    cbc.fit(x_train,y_train, eval_set = (x_valid,y_valid), early_stopping_rounds = 30)
    
    
    result2 += cbc.predict_proba(test2)/10 
        



Learning rate set to 0.05935
0:	learn: 0.6358364	test: 0.6363871	best: 0.6363871 (0)	total: 61.1ms	remaining: 1m 1s
100:	learn: 0.2558150	test: 0.2717625	best: 0.2717450 (95)	total: 610ms	remaining: 5.43s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.271201674
bestIteration = 156

Shrink model to first 157 iterations.
Learning rate set to 0.05935
0:	learn: 0.6359979	test: 0.6360439	best: 0.6360439 (0)	total: 5.79ms	remaining: 5.78s
100:	learn: 0.2564260	test: 0.2710823	best: 0.2710823 (100)	total: 535ms	remaining: 4.76s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.2708344128
bestIteration = 116

Shrink model to first 117 iterations.
Learning rate set to 0.05935
0:	learn: 0.6360098	test: 0.6363778	best: 0.6363778 (0)	total: 6.64ms	remaining: 6.63s
100:	learn: 0.2552803	test: 0.2767824	best: 0.2767361 (98)	total: 535ms	remaining: 4.77s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.2762752554
bestIteration = 110

Shrink mo

In [21]:
# cbc = CatBoostClassifier(verbose = 100)
# cbc.fit(train2,train['Y_LABEL'])
# #cbc.fit(x_train,y_train,eval_set = (x_valid,y_valid))
# result2 = cbc.predict_proba(test2)

In [22]:
#threshold 확정
result2_arr = (result2[:,1]>0.151)
result2_arr

array([False, False, False, ..., False, False, False])

In [23]:
sub = pd.read_csv('/kaggle/input/dacon-data/sample_submission.csv')
sub['Y_LABEL'] = result2_arr     #threshold 설정: 확율 0.151이상일때 True로 분류
sub['Y_LABEL'] = sub['Y_LABEL'].astype(int)
sub.to_csv('sub2.csv',index = 0)
sub

,ID,Y_LABEL
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,1
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [24]:
sub['Y_LABEL'].value_counts() #과제 : 점수 가장 잘 나오는 비율찾기

0    5497
1     544
Name: Y_LABEL, dtype: int64

In [25]:
#feature 중요도
pd.Series(cbc.feature_importances_,index = train2.columns).sort_values(ascending = False)

ANONYMOUS_1            17.631127
YEAR                   12.772401
ANONYMOUS_2             9.048825
ZN                      4.952974
FE                      4.766207
V40                     4.288384
std_y                   4.159777
mean_y                  4.150381
sum_y                   4.018739
CU                      3.787757
PQINDEX                 3.617310
CR                      3.253066
MO                      3.056127
count_y                 3.016188
max_y                   2.951654
median_y                2.129053
TI                      1.578886
sum_x                   1.374459
max_x                   1.263841
median_x                1.248400
NI                      1.151490
AG                      0.962136
COMPONENT_ARBITRARY     0.961939
MN                      0.854565
count_x                 0.821963
mean_x                  0.729336
std_x                   0.691250
V                       0.312220
H2O                     0.252698
CO                      0.196845
dtype: flo